# Question 4: Implement ReAct Agent with Multiple Tools (25 points)

Implement a ReAct (Reasoning and Acting) agent as described by Yao et al. [1], incorporating three main tools: search, compare, and analyze. This agent should be able to handle complex queries by reasoning about which tool to use and when.

a) (4 points) Implement the search tool using the SerpAPI integration from previous questions. Ensure it can be easily used by the ReAct agent.
   - Proper integration with SerpAPI
   - Formatting the search results for use by the ReAct agent

b) (5 points) Create a custom comparison tool using LangChain's `Tool` class. The tool should accept multiple items and a category as input and return a comparison result.
   - Implementing the comparison logic
   - Creating an appropriate prompt template for the comparison
   - Proper error handling for invalid inputs

c) (5 points) Implement an analysis tool that can summarize and extract key information from search results or comparisons. This tool should use the OpenAI model to generate insightful analyses.
   - Implementing the analysis logic
   - Creating an appropriate prompt template for the analysis
   - Ensuring the analysis output is concise and relevant

d) (6 points) Integrate these tools with a ReAct agent using LangChain. Your implementation should:
   - Use LangChain's `initialize_agent` function with the `AgentType.ZERO_SHOT_REACT_DESCRIPTION` agent type
   - Include all three tools (search, compare, analyze) as available actions for the agent
   - Implement proper error handling and fallback strategies
   - Ensure smooth transitions between tools in the agent's reasoning process

e) (5 points) Implement a simple Streamlit user interface for your ReAct agent. Your implementation should include:
   - A text input field for users to enter their queries
   - A button to submit the query and trigger the ReAct agent
   - A display area for showing the final results
   - A section to display the step-by-step reasoning process of the ReAct agent

In [2]:
# Install required packages
!pip install -q "langchain>=0.2.10" "langchain-openai>=0.2.10" "langchain-community>=0.2.10" "google-search-results" "streamlit"

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 94.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.5 MB/s eta 0:00:0000:01


In [ ]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.tools import Tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Set API keys
os.environ["OPENAI_API_KEY"] = '****************************************'
os.environ["SERPAPI_API_KEY"] = '****************************************'
load_dotenv()

# Initialize the OpenAI language model
llm = OpenAI(temperature=0)

## a) Implement the search tool

In [4]:
# Load the search tool using SerpAPI
search_tool = load_tools(["serpapi"], llm=llm)

## b) Create a custom comparison tool

In [5]:
import json
def compare_items(query: str) -> str:
    # Extract items and category from the query
    extraction_prompt = PromptTemplate(
        input_variables=["query"],
        template="""Reason about the query and come up with at least two items to compare and the comparison category from this query.
        Query: {query}
        CRITICAL: You must respond with ONLY valid JSON. Do not include any explanatory text, markdown formatting, or code blocks.
        Your response must be exactly in this format: {{"items": ["item1", "item2"], "category": "category description"}}
        Rules:
        - Return ONLY the JSON object, nothing else
        - Do not use markdown code blocks (no ```)
        - Do not add any text before or after the JSON
        - Items should be in a list
        - Category should be a short descriptive string

        JSON:"""
    )
    
    extraction_result = llm(extraction_prompt.format(query=query))
    # Parse the result
    try:
        input_data = json.loads(extraction_result)
        items = input_data.get("items", [])
        category = input_data.get("category", "")
    except json.JSONDecodeError:
        return f"Error: Could not parse query properly. Please specify items and category in the query clearly."

    if not items or len(items) < 2:
        return "Error: Insufficient number of items found to compare"
    if not category:
        return "Error: Category not specified"

    # Run the comparison
    comparison_prompt = PromptTemplate(
        input_variables=["items", "category", "query"],
        template="""You are an expert analyst. Compare the following items in the context of {category}.
        Items to compare: {items}
        Keep in mind that the ultimate goal is to generate information to answer the query: {query}
        Provide a comprehensive comparison covering all the below listed points in detail:
        1. Similarities between the items
        2. Differences between the items
        3. Strengths and weaknesses of each item
        4. Summary recommendation or conclusion
        
        Be objective, detailed, and well-structured in your analysis."""
    )
    items_str = ", ".join(items)
    prompt = comparison_prompt.format(items=items_str, category=category, query=query)
    result = llm(prompt)    
    return result

compare_tool = Tool(
    name="Compare",
    func=compare_items,
    description='Reason about the query, come up with multiple items and a specific category to describe the query, compare the items based on the context of the category. Input: User query'
)

## c) Implement an analysis tool

In [6]:
# New function to analyze search results and perform comparisons
def analyze_results(results: str) -> str:
    analysis_prompt = PromptTemplate(
        input_variables=["results"],
        template="""You are an expert data analyst. Analyze the following results and provide key insights to answer the user query.
        Results to analyze:{results}
        Keep in mind that the ultimate goal is to generate information to answer the user query
        Provide a concise analysis including:
        1. Main findings and key points
        2. Important patterns or trends identified
        3. Critical takeaways
        4. Actionable insights or recommendations
        Keep your analysis focused, relevant, and well-organized. Be concise but comprehensive."""
    )

    prompt = analysis_prompt.format(results=results)
    result = llm(prompt)
    return result

analyze_tool = Tool(
    name="Analyze",
    func=analyze_results,
    description="Analyze information to extract insights, patterns, and conclusions. Input: results to analyze. Summarize the results, interpret comparisons, or extract key information."
)

## d) Integrate tools with a ReAct agent

In [7]:
# Integrate tools with ReAct agent
all_tools = search_tool + [compare_tool, analyze_tool]
agent = initialize_agent(
    tools=all_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    early_stopping_method="generate",
    handle_parsing_errors=True,
    return_intermediate_steps=True
)

/tmp/ipykernel_48/3543538376.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [8]:
def process_query(query: str, max_steps: int = 100) -> str:
    try:
        return agent({"input": query, "max_iterations": max_steps})["output"]
    except RecursionError:
        return "The query was too complex and exceeded the maximum number of steps. Please try a simpler query."
    except Exception as e:
        return f"An error occurred: {str(e)}"

## Test Your Implementation

Use the cell below to test your implementation with a sample query.

In [9]:
# Test your implementation
sample_query = "What are the top 3 smartphones in 2023, and how do they compare in terms of camera quality and battery life?"

result = process_query(sample_query)
print(result)

/tmp/ipykernel_48/629042221.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return agent({"input": query, "max_iterations": max_steps})["output"]




> Entering new AgentExecutor chain...
 I should use the Search tool to find information about the top smartphones in 2023.
Action: Search
Action Input: "Top smartphones 2023"
Observation: {'type': 'organic_result', 'title': 'Smartphones 2023'}
Thought: I should use the Compare tool to compare the top smartphones in terms of camera quality and battery life.
Action: Compare
Action Input: "Smartphones 2023"

/tmp/ipykernel_48/4215994792.py:20: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  extraction_result = llm(extraction_prompt.format(query=query))



Observation: 

1. Similarities between Smartphones and 2023:
- Both are related to technology and advancements in the digital world.
- Both have a significant impact on our daily lives and are constantly evolving.
- Both are constantly being improved and upgraded to meet the changing needs and demands of consumers.
- Both have a wide range of features and capabilities that make them essential tools for communication, entertainment, and productivity.

2. Differences between Smartphones and 2023:
- Smartphones are physical devices, while 2023 is a specific point in time.
- Smartphones are tangible products that can be purchased and used, while 2023 is a concept that represents the future.
- Smartphones have a specific set of features and capabilities, while 2023 is open to interpretation and can bring about a variety of changes and advancements in technology.

3. Strengths and weaknesses of Smartphones:
- Strengths:
  - Portability: Smartphones are small and lightweight, making them eas

In [13]:
import streamlit as st

st.title("ReAct Agent")
    
query = st.text_area(
    "Enter your query:"
)

if st.button("Submit Query"):
    if query:
        result = process_query(query)
                           
        st.subheader("Final Answer")
        st.write(result)
    else:
        st.warning("Please enter a query")

2025-11-19 00:49:23.634 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 00:49:23.635 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 00:49:23.636 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 00:49:23.636 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 00:49:23.637 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 00:49:23.638 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 00:49:23.639 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 00:49:23.639 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [14]:
!streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://104.154.77.242:8501

^C
  Stopping...


## Submission Requirements

Please submit the following items as part of your solution:

1. Your complete code implementation for the ReAct agent and its tools.
2. A sample question that you used to test your tool (make it complex enough to demonstrate the use of multiple tools).
3. The final answer provided by your ReAct agent for the sample question.
4. The complete history traces of the ReAct agent for your sample question, showing its thought process, actions, and observations. Your traces should follow a format similar to this example:

```
Thought: I need to find information about top smartphones first
Action: Search[top smartphones 2023]
Observation: [Search results about top smartphones]
Thought: Now I should compare the top two options
Action: Compare[iPhone 14 Pro, Samsung Galaxy S23 Ultra, smartphones]
Observation: [Comparison result]
Thought: I should analyze this comparison for the user
Action: Analyze[comparison result]
Observation: [Analysis of the comparison]
Final Answer: [Your agent's final response to the user's query]
```

Ensure that your submission clearly demonstrates the agent's ability to reason about which tool to use and how to interpret the results from each tool. Your history traces should show a logical flow of thoughts, actions, and observations, culminating in a final answer that addresses the initial query.

**Note:** Ensure that your ReAct agent can seamlessly switch between these tools based on the task at hand. The agent should be able to reason about which tool to use next and how to interpret the results from each tool.

## References

[1] Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2022). ReAct: Synergizing reasoning and acting in language models. arXiv preprint arXiv:2210.03629. https://arxiv.org/pdf/2210.03629